# Solutions to sheet 4

Import the necessary libraries: `numpy`, `matplotlib`. We can use the `as` keyword to define shortcuts to access the libraries (example: `import numpy as np` means we can access numpy functions with the `np.` syntax).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import scipy.stats as stats

In [ ]:
plt.rcParams.update({
  'figure.figsize': [8.0, 5.0],
  'xtick.major.width': 1.5,
  'xtick.major.size': 10.0,
  'xtick.minor.size': 5.0,
  'ytick.major.width': 1.5,
  'ytick.major.size': 10.0,
  'ytick.minor.size': 5.0,
  'font.size': 16,
})

Fix some of the physics parameters we will use.

In [ ]:
m_top = 173       # in GeV
gamma_top = 1.33  # in GeV

## Implement Cauchy and Gauss distributions

This implements a Cauchy distribution according to:

$$
f(x) = \frac{1}{\pi\gamma} \cdot \frac{\gamma^2}{(x - x_0)^2 + \gamma^2}
$$

In [ ]:
def Cauchy(x, x_0, gamma):
    """Implementation of a Cauchy distribution.
    
    Args:
      x: the point at which to evaluate
      x_0: the central value of the distribution
      gamma: the half width at half maximum
    
    Returns:
      function evaluated at point x
    """
    pref = 1 / np.pi / gamma
    return pref * np.power(gamma, 2) / (np.power(x - x_0, 2) + np.power(gamma, 2))

This implements a Gauss distribution according to:

$$
f(x) = \frac{1}{\sigma \sqrt{2 \pi}} \cdot \exp \left( - \frac{(x - \mu)^2}{ 2 \sigma^2} \right)
$$

In [ ]:
def Gauss(x, mu, sigma):
    """Implementation of a Gauss distribution.
    
    Args:
      x: the point at which to evaluate
      mu: the central value of the distribution
      sigma: the standard deviation of the distribution
    
    Returns:
      function evaluated at point x
    """
    pref = 1 / sigma / np.sqrt(2 * np.pi)
    return pref * np.exp(- np.power(x - mu, 2) / 2 / sigma / sigma)

## Plot the distribution

Let's plot 200 points in the interval $[170, 176]$.

In [ ]:
n_points = 200
interval = (170, 176)

In [ ]:
xfunc = np.linspace(*interval, n_points)

In [ ]:
yfunc = Cauchy(xfunc, m_top, gamma_top/2)

In [ ]:
plt.plot(xfunc, yfunc, label="Cauchy distribution", lw=4)
plt.xlim(*interval)
plt.xlabel("mass [GeV]")
plt.legend()
plt.tight_layout()

## Accept-reject method

Use a random number generator to generate 2000 random points following a uniform distribution scaled with the pre-factor $0.3$.

In [ ]:
gen = np.random.default_rng()

In [ ]:
n_samples = 2000

In [ ]:
x = gen.uniform(*interval, n_samples)

In [ ]:
y = gen.uniform(0, 0.5, n_samples)

In [ ]:
plt.plot(xfunc, np.full(n_points, 0.5), label="Uniform envelope", lw=4)
plt.scatter(x, y, label="Sample points", marker=".")
plt.plot(xfunc, yfunc, label="Cauchy distribution", lw=4)
plt.xlim(*interval)
plt.ylim([0, 0.75])
plt.xlabel("mass [GeV]")
plt.legend()
plt.tight_layout()

We cannot distinguish accepted and rejected points yet. So let's apply the Von Neumann criterion.

In [ ]:
accepted = (y <= Cauchy(x, m_top, gamma_top/2))

In [ ]:
x_accepted = x[accepted]
y_accepted = y[accepted]

In [ ]:
x_rejected = x[np.invert(accepted)]
y_rejected = y[np.invert(accepted)]

In [ ]:
plt.plot(xfunc, np.full(n_points, 0.5), label="Uniform envelope", lw=4)
plt.plot(xfunc, yfunc, label="Cauchy distribution", lw=4)
plt.scatter(x_rejected, y_rejected, label="Rejected points", marker=".")
plt.scatter(x_accepted, y_accepted, label="Accepted points", marker=".")
plt.xlim(*interval)
plt.xlabel("mass [GeV]")
plt.legend()
plt.tight_layout()

In [ ]:
print("Acceptance rate: {}".format(len(x_accepted) / len(x)))

## accept-reject method with Gauss

Let's repeat the same exercise, but this time following a Gauss distribution.

In [ ]:
gfunc = 2 * Gauss(xfunc, m_top, gamma_top)

In [ ]:
plt.plot(xfunc, gfunc, label="Gauss distribution", lw=4)
plt.plot(xfunc, yfunc, label="Cauchy distribution", lw=4)
plt.xlim(*interval)
plt.xlabel("mass [GeV]")
plt.legend()
plt.tight_layout()

In [ ]:
n_samples = 2000

In [ ]:
x = gen.uniform(*interval, n_samples)

In [ ]:
y = 2 * Gauss(x, m_top, gamma_top) * gen.uniform(0, 1, n_samples)

In [ ]:
plt.plot(xfunc, gfunc, label="Gauss distribution", lw=4)
plt.plot(xfunc, yfunc, label="Cauchy distribution", lw=4)
plt.scatter(x, y, label="Sample points", marker=".")
plt.xlim(*interval)
plt.xlabel("mass [GeV]")
plt.legend()
plt.tight_layout()

In [ ]:
accepted = (y <= Cauchy(x, m_top, gamma_top/2))

In [ ]:
x_accepted = x[accepted]
y_accepted = y[accepted]

In [ ]:
x_rejected = x[np.invert(accepted)]
y_rejected = y[np.invert(accepted)]

In [ ]:
plt.plot(xfunc, gfunc, label="Gauss distribution", lw=4)
plt.plot(xfunc, yfunc, label="Cauchy distribution", lw=4)
plt.scatter(x_rejected, y_rejected, label="Rejected points", marker=".")
plt.scatter(x_accepted, y_accepted, label="Accepted points", marker=".")
plt.xlim(*interval)
plt.xlabel("mass [GeV]")
plt.legend()
plt.tight_layout()

In [ ]:
print("Acceptance rate: {}".format(len(x_accepted) / (len(x))))

Why is the acceptance rate still so low? And what about the point density? Is it constant across the shape of the envelope? If not, why not?


## Trying to fill the points into a histogram

Let's try to fill the points into a histogram.

In [ ]:
n_bins = 30

In [ ]:
plt.plot(xfunc, yfunc, label="Cauchy distribution", lw=4)
plt.plot(xfunc, gfunc, label="Gauss distribution", lw=4)
plt.hist(x_accepted, label="Accepted points", bins=n_bins, alpha=0.5, density=True)
plt.xlim(*interval)
plt.xlabel("mass [GeV]")
plt.legend()
plt.tight_layout()

Why is the shape not reflected correctly in the histogram? What did we forget? We need to remember that the point density in $x$ was generated according to a uniform distribution, but we chose the Gauss function is an envelope! So we need to apply additional event weights.

In [ ]:
weights_accepted = 2 * Gauss(x_accepted, m_top, gamma_top)

In [ ]:
weights_accepted = weights_accepted * n_bins / n_samples

In [ ]:
plt.plot(xfunc, yfunc, label="Cauchy distribution", lw=4)
plt.plot(xfunc, gfunc, label="Gauss distribution", lw=4)
contents, bin_edges, _ = plt.hist(x_accepted, label="Accepted points", weights=weights_accepted, bins=n_bins, alpha=0.5)
plt.xlim(*interval)
plt.xlabel("mass [GeV]")
plt.legend()
plt.tight_layout()

In [ ]:
bin_width = (bin_edges[-1] - bin_edges[0]) / n_bins

In [ ]:
np.sum(contents) * bin_width

In [ ]:
integrate.quad(lambda x: Cauchy(x, m_top, gamma_top/2), *interval)[0]

## How can we avoid the problem of uneven point densities?

The issue is that we sampled $x \in [170, 176]$ uniformly without caring about the shape of the Gaussian envelope distribution. To achieve uniform point density, we need to consider the cumulative distribution function (CDF) of the Gaussian distribution. This leads us to the inversion method.

Just for reference: the code below repeats the same exercise as above, but this time the $x$ values are sampled by drawing from a uniform distribution in $[0, 1]$ and then projecting these values on the inverse of the CDF of the Gaussian distribution. This gives us uniform point density.

In [ ]:
n_samples = 2000

In [ ]:
# This is the "percent-point function" (quantile function) of the Gaussian distribution,
# i.e. the inverse of its cumulative density function.
x = stats.norm.ppf(gen.uniform(0, 1, n_samples), loc=m_top, scale=gamma_top)

In [ ]:
y = 2 * Gauss(x, m_top, gamma_top) * gen.uniform(0, 1, n_samples)  # Same as before!

In [ ]:
plt.plot(xfunc, gfunc, label="Gauss distribution", lw=4)
plt.plot(xfunc, yfunc, label="Cauchy distribution", lw=4)
plt.scatter(x, y, label="Sample points", marker=".")
plt.xlim(*interval)
plt.xlabel("mass [GeV]")
plt.legend()
plt.tight_layout()

In [ ]:
accepted = (y <= Cauchy(x, m_top, gamma_top/2))

In [ ]:
x_accepted = x[accepted]
y_accepted = y[accepted]

In [ ]:
x_rejected = x[np.invert(accepted)]
y_rejected = y[np.invert(accepted)]

In [ ]:
plt.plot(xfunc, gfunc, label="Gauss distribution", lw=4)
plt.plot(xfunc, yfunc, label="Cauchy distribution", lw=4)
plt.scatter(x_rejected, y_rejected, label="Rejected points", marker=".")
plt.scatter(x_accepted, y_accepted, label="Accepted points", marker=".")
plt.xlim(*interval)
plt.xlabel("mass [GeV]")
plt.legend()
plt.tight_layout()

In [ ]:
print("Acceptance rate: {}".format(len(x_accepted) / (len(x))))

As we can see, this leads also to a higher acceptance rate. Let's fill the histogram again. Since we have achieved uniform point density already, we don't need to apply any weights.

In [ ]:
n_bins = 30

In [ ]:
plt.plot(xfunc, yfunc, label="Cauchy distribution", lw=4)
plt.plot(xfunc, gfunc, label="Gauss distribution", lw=4)
plt.hist(x_accepted, label="Accepted points", bins=n_bins, range=interval, alpha=0.5, density=True)
plt.xlim(*interval)
plt.xlabel("mass [GeV]")
plt.legend()
plt.tight_layout()

In [ ]:
bin_width = (bin_edges[-1] - bin_edges[0]) / n_bins

In [ ]:
np.sum(contents) * bin_width

In [ ]:
integrate.quad(lambda x: Cauchy(x, m_top, gamma_top/2), *interval)[0]